In [ ]:
import spacy
from google.colab import drive
from pathlib import Path
from multiprocessing import Pool
import pandas as pd
import regex as re
import spacy

drive.mount('/content/drive')
!python -m spacy download pl_core_news_sm

materials_path = Path("/run/user/1000/gvfs/google-drive:host=gmail.com,user=przemek.7678/Shared drives/SKN AI FUW /Projekt z Dagmarą Mateją/NLP materials")
corpus_path = Path("/run/user/1000/gvfs/google-drive:host=gmail.com,user=przemek.7678/Shared drives/SKN AI FUW /Projekt z Dagmarą Mateją/N")
nlp = spacy.load('pl_core_news_sm')

Mounted at /content/drive
2023-11-12 10:53:59.657486: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-12 10:53:59.657557: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-12 10:53:59.657598: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-12 10:54:01.356441: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 67.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pl_core_news_sm')


In [ ]:
nlp = spacy.load('pl_core_news_sm')
stopwords = pd.read_csv( materials_path / 'polish_stopwords.txt', header=None)
STOPWORDS = set([word.rstrip() for word in stopwords[0]])
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;\.-]')
REPLACE_EXCESS_SPACES_RE = re.compile('\s\s+')
BAD_SYMBOLS_RE = re.compile('[^AaĄąBbCcĆćDdEeĘęFfGgHhIiJjKkLlŁłMmNnŃńOoÓóPpRrSsŚśTtUuWwYyZzŹźŻż1234567890 ]')


def lemmatizer(text):
  doc = nlp(text)
  return " ".join([token.lemma_ for token in doc])

def clean_text(text):
    text = text.lower()
    text = REPLACE_BY_SPACE_RE.sub(' ', text)
    text = REPLACE_EXCESS_SPACES_RE.sub(' ', text)
    text = BAD_SYMBOLS_RE.sub('', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS)
    if text == None:
      text == "None None None"
    return text

def preprocess(df):
    df = df[~df['text'].isna()]
    df['title'] = df['title'].fillna("None")
    print("Cleanig...")
    df['clean_text'] = df['text'].apply(clean_text)
    df['clean_title'] = df['title'].apply(clean_text)
    print("Lemmatizing...")
    df['clean_title'] = df['clean_title'].apply(lemmatizer)
    df['clean_text'] = df['clean_text'].apply(lemmatizer)
    return df

def colab_preprocessing(link):
  df = pd.read_csv(link, index_col = "id")
  if "clean_text" in df.columns.values:
    print(link)
    return
  if (df['source'] == "WPROST").all() or (df['source'] == "Newsweek").all():
      df['title'] = df['title_main'] + df['title_sub']
  df = preprocess(df)
  print(link)
  df.to_csv(link)

In [ ]:
links_gw = [corpus_path / "Gazeta Wyborcza (do 2018 r.)" / "data" / f"wyborcza_{i}.csv" for i in range(22)]
links_wprost = [corpus_path / "Wprost 2015-2022" / "data" / f"Wprost.csv"]
links_newsweek = [corpus_path / "Newsweek 2015-2022" / 'data' / f"newsweek.csv"]
links_dorzeczy = [corpus_path / "DoRzeczy" / "data" / "dorzeczy.csv"]
links_polityka = [corpus_path / "Polityka" / "data" / f"polityka_{i}.csv" for i in range(5)]
links_gpc = [corpus_path / "Gazeta Polska Codziennie" / "data" / f"gpc_{i}.csv" for i in range(13)]
links_rzepa = [corpus_path / "Rzeczpospolita 2015-2022" / "data"/ f"rzepa_{i}.csv" for i in range(22)]
links_dataset = [materials_path / "dataset.csv"]
links_wpolityce = [corpus_path / "wPolityce" / "wPolityce.csv" ]


corpus_links = {
    "newsweek" : links_newsweek,
    "rzepa" : links_rzepa,
    "gpc" : links_gpc,
    "wprost" : links_wprost,
    "polityka" : links_polityka,
    "dorzeczy" : links_dorzeczy,
    "dataset" : links_dataset,
    "wyborcza" : links_gw,
    "wpolityce" : links_wpolityce
}

In [ ]:
for links in corpus_links.keys():
    with Pool(3) as pool:
        pool.map(colab_preprocessing, links)